# Stochastic event set generation in openquake.hazardlib

<tt>

LICENSE

Copyright (c) 2014, GEM Foundation, G. Weatherill, M. Pagani, D. Monelli.

The notebook is free software: you can redistribute
it and/or modify it under the terms of the GNU Affero General Public
License as published by the Free Software Foundation, either version
3 of the License, or (at your option) any later version.

You should have received a copy of the GNU Affero General Public License
along with OpenQuake. If not, see <http://www.gnu.org/licenses/>

DISCLAIMER
 
The notebook provided herein is released as a prototype
implementation on behalf of scientists and engineers working within the GEM
Foundation (Global Earthquake Model).

It is distributed for the purpose of open collaboration and in the
hope that it will be useful to the scientific, engineering, disaster
risk and software design communities.

The software is NOT distributed as part of GEM's OpenQuake suite
(http://www.globalquakemodel.org/openquake) and must be considered as a
separate entity. The software provided herein is designed and implemented
by scientific staff. It is not developed to the design standards, nor
subject to same level of critical review by professional software
developers, as GEM's OpenQuake software suite.

Feedback and contribution to the software is welcome, and can be
directed to the hazard scientific staff of the GEM Model Facility
(hazard@globalquakemodel.org).

The notebook is therefore distributed WITHOUT ANY WARRANTY; without
even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR
PURPOSE. See the GNU General Public License for more details.

The GEM Foundation, and the authors of the software, assume no liability for
use of the software.
</tt>

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline

from openquake.hazardlib.source import AreaSource
from openquake.hazardlib.mfd import TruncatedGRMFD
from openquake.hazardlib.scalerel import WC1994
from openquake.hazardlib.geo import Point, NodalPlane, Polygon
from openquake.hazardlib.pmf import PMF
from openquake.hazardlib.tom import PoissonTOM
from openquake.hazardlib.calc.stochastic import stochastic_event_set

import matplotlib
import numpy
import os
from matplotlib import pyplot
from matplotlib import patches
from mpl_toolkits.basemap import Basemap
from obspy.imaging.beachball import beach
from IPython.display import Image

In [ ]:
# beach ball reference width
BB_WIDTH = 40000

# magnitude bins
MAG_BINS = numpy.array([4., 5., 6., 7., 8., 9.])
MAG_BB_WIDTHS = numpy.array([BB_WIDTH * 0.5, BB_WIDTH, BB_WIDTH * 1.5, BB_WIDTH * 2., BB_WIDTH * 2.5])
MAG_BB_COLORS = numpy.array(['r', 'g', 'b', 'c', 'm'])

In [ ]:
def get_map_projection(src):
    """
    Return map projection specific to source.
    """
    # extract rupture enclosing polygon (considering a buffer of 10 km)
    rup_poly = src.get_rupture_enclosing_polygon(10.)
    min_lon = numpy.min(rup_poly.lons)
    max_lon = numpy.max(rup_poly.lons)
    min_lat = numpy.min(rup_poly.lats)
    max_lat = numpy.max(rup_poly.lats)
    
    # create map projection
    m = Basemap(projection='merc', llcrnrlat=min_lat, urcrnrlat=max_lat,
                llcrnrlon=min_lon, urcrnrlon=max_lon, resolution='l')

    return min_lon, max_lon, min_lat, max_lat, m

# Stochastic event set generated by an area source

In [ ]:
# time span for stochastic event set generation
time_span = 1000.

# define area source
src = AreaSource(
    source_id='1',
    name='area',
    tectonic_region_type='Active Shallow Crust',
    mfd=TruncatedGRMFD(min_mag=5., max_mag=6.5, bin_width=0.2, a_val=4.45, b_val=0.98),
    rupture_mesh_spacing=2.,
    magnitude_scaling_relationship=WC1994(),
    rupture_aspect_ratio=1.,
    temporal_occurrence_model=PoissonTOM(time_span),
    upper_seismogenic_depth=2.,
    lower_seismogenic_depth=12.,
    nodal_plane_distribution=PMF([(0.125, NodalPlane(strike=0, dip=90, rake=0)),
                                  (0.125, NodalPlane(strike=45, dip=90, rake=0)),
                                  (0.125, NodalPlane(strike=90, dip=90, rake=0)),
                                  (0.125, NodalPlane(strike=135, dip=90, rake=0)),
                                  (0.0625, NodalPlane(strike=0, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=45, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=90, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=135, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=180, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=225, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=270, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=315, dip=50, rake=90))]),
    hypocenter_distribution=PMF([(1, 7.)]),
    polygon=Polygon([Point(133.5, -22.5), Point(133.5, -23.0), Point(130.75, -23.75), Point(130.75, -24.5),
                     Point(133.5, -26.0), Point(133.5, -27.0), Point(130.75, -27.0), Point(128.977, -25.065),
                     Point(128.425, -23.436), Point(126.082, -23.233), Point(125.669, -22.351), Point(125.4, -20.5),
                     Point(125.75, -20.25), Point(126.7, -21.25), Point(128.5, -21.25), Point(129.25, -20.6),
                     Point(130.0, -20.6), Point(130.9, -22.25), Point(133.0, -22.0), Point(133.5, -22.5)]),
    area_discretization=20.
)

In [ ]:
# generate two different stochastic event sets from the area source
numpy.random.seed(123)

# generate event and store them in lists
ses1 = list(stochastic_event_set([src]))
ses2 = list(stochastic_event_set([src]))

In [ ]:
def write_ruptures_file(src, fname):
    fou = open(fname, 'w')   
    for rup in src.iter_ruptures():
        surf = rup.surface
        fou.write('> -Z{:.2f}\n'.format(rup.mag))
        if hasattr(surf, 'top_left'):
            lons, lats = get_planar_surface_boundary(surf)
        else:
            lons, lats = get_mesh_boundary(surf.mesh)

        for lo, la in zip(lons, lats):
            fou.write('{:.4f} {:.4f}\n'.format(lo, la))
    fou.close()

    
def write_fm_file(ses, fname):
    fou = open(fname, 'w')
    for rup in ses:
        strike = rup.surface.get_strike()
        dip = rup.surface.get_dip()
        rake = rup.rake
        lo = rup.hypocenter.longitude
        la = rup.hypocenter.latitude
        de = rup.hypocenter.depth
        mag = rup.mag
        fmt = '{:.4f} {:.4f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}\n'
        fou.write(fmt.format(lo, la, de, strike, dip, rake, mag))
    fou.close()

In [ ]:
fname1 = '/tmp/ses01.txt'
write_fm_file(ses1, fname1)
fname2 = '/tmp/ses02.txt'
write_fm_file(ses2, fname2)
figname = '/tmp/ses.ps'

In [ ]:
%%bash -s "$fname1" "$fname2" "$figname" 

CPT='/tmp/tmp.cpt'
GRD='/tmp/tmp.grd'
TMP='/tmp/tmp.xyz'

cat $1 $2 > $TMP
EXTENT=$(gmt info -I.5 -D $TMP)

gmt set MAP_FRAME_TYPE = PLAIN
gmt set MAP_GRID_CROSS_SIZE_PRIMARY = 0.2i
gmt set MAP_FRAME_TYPE = PLAIN
gmt set FONT_TITLE = 12p
gmt set FONT_LABEL = 10p

gmt psbasemap $EXTENT -Jm2c -K -B1/1WSne > $3
gmt psmeca $1 -R -J -K -O -Sa0.25 >> $3
gmt psmeca $2 -R -J -K -O -Sa0.25 -Gred >> $3
gmt pscoast -R -J -N1 -O -W1 -Dh >> $3

gmt psconvert $3 -Tg -A -P 
rm gmt.*

In [ ]:
Image(filename=os.path.splitext(figname)[0]+'.png', width=600)